In [ ]:
import subprocess
import os
import time
import numpy as np

import gzip
import json

from utils_mobility_demand import load_route_file_in_dict, create_xml_vehicles

In [ ]:
def generate_intervals(N, S):
    
    # each interval is [a, b)
    
    interval_size = N // S
    intervals = []
    for i in range(S):
        start = i * interval_size
        end = (i + 1) * interval_size - 1 if i < S - 1 else N - 1
        intervals.append((start, end+1))
    return intervals


def save_dict_to_gzipped_json(data, filename):
    with gzip.open(filename, 'wt') as gzipped_file:
        json.dump(data, gzipped_file)

def load_dict_from_gzipped_json(filename):
    with gzip.open(filename, 'rt') as gzipped_file:
        data = json.load(gzipped_file)
    return data

In [ ]:
city = "milan"
N = 5000
njobs = 5

w = 1

# ----- #

demand_file = f"../data/{city}/mobility_demand/N{N}/dict_mobility_demand_{city}_N{N}.json"

road_network_path = f"../data/road_networks/sumo_road_network_{city}.net.xml"

# TMP
result_folder = f"../data/{city}/mydua_tmp/"
result_folder_chunks = f"{result_folder}tmp_mydua_w{str(w).replace('.','p')}/"

print(f"w={w}", road_network_path)

In [ ]:
intervals = generate_intervals(N, njobs)
print(intervals)

In [ ]:
# Record the start time
start_time = time.time()

processes = []

np.random.seed()
seed = np.random.randint(0, 1e7)

print(f"Random seed: {seed}")

for interval in intervals:
    
    options = f"-c {city} -N {N} -w {w} --seed {seed} --road-file {road_network_path} -d {demand_file} --ind-from {interval[0]} --ind-to {interval[-1]}"

    command_list = ['python', "worker_mydua.py"] + options.split(" ")

    script = subprocess.Popen(command_list)#, stdout=subprocess.DEVNULL)
    processes.append(script)

    
print("Waiting for all the processes to end...")
for process in processes:
    process.wait()


end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed Time:", elapsed_time, "seconds")   

# merging all the chunks

list_dict_chunks = [load_dict_from_gzipped_json(result_folder_chunks+d) for d in os.listdir(result_folder_chunks) if ".json" in d]
print(os.listdir(result_folder_chunks))

assert len(list_dict_chunks) == njobs
merged_dict = {k: v for d in list_dict_chunks for k, v in d.items()}
assert len(merged_dict) == N

with open(demand_file, 'r') as f:
    dict_demand = json.load(f)["demand"]
    
    
# Create the dict for the routed paths

dict_routed_paths = {}

for vid in dict_demand:
    
    vid_int_str = vid.split("_")[-1]
    
    # departure time
    dep_time = dict_demand[vid]["time"]

    # edge list
    edge_list = merged_dict[vid_int_str]
    
    dict_routed_paths[vid] = {"edges": edge_list, "time":dep_time}
    
    
# SAVE THE ROUTED PATHS
if w > 1:
    filename_routed_paths = f"../data/{city}/routed_paths/N{N}/routed_paths_{city}_N{N}_myduaw{w}.rou.xml"
else:
    filename_routed_paths = f"../data/{city}/routed_paths/N{N}/routed_paths_{city}_N{N}_IGfastest.rou.xml"
    
if not os.path.exists(f"../data/{city}/routed_paths/N{N}/"):
    os.makedirs(f"../data/{city}/routed_paths/N{N}/", exist_ok=True)

create_xml_vehicles(dict_routed_paths, filename_routed_paths, lane_best=True, compress=True, text_comment=f"Seed {seed}")


